# Procesamiento paralelo en Python

El número de librerías y paquetes para el procesamiento paralelo en Python es enorme. Consulta este enlace para obtener una visión general (https://wiki.python.org/moin/ParallelProcessing).

En esta sesión presentamos multiprocessing, uno de los frameworks más importantes para implementar aplicaciones paralelas en Python.


# Módulo multiprocessing de Python

El módulo `multiprocessing` (https://docs.python.org/3/library/multiprocessing.html) es una biblioteca potente y versátil incluida en la distribución estándar de Python, diseñada para facilitar la ejecución concurrente/paralela mediante la creación de múltiples procesos. Proporciona una interfaz de alto nivel para la creación y gestión de procesos, así como una amplia variedad de herramientas de bajo nivel para construir sistemas complejos de comunicación y sincronización entre procesos.

A diferencia de los hilos, que operan dentro del mismo espacio de memoria y están sujetos al Global Interpreter Lock (GIL), `multiprocessing` crea procesos separados con espacios de memoria individuales. Esto permite un paralelismo real en procesadores multinúcleo. El caso ideal sería ejecutar un proceso por cada procesador físico; de esta forma extraeríamos el máximo potencial de paralelismo de nuestro procesador.

El módulo incluye herramientas para:

- Crear y gestionar procesos.
- Compartir datos entre procesos mediante memoria compartida.
- Establecer canales de comunicación como `Pipes` y `Queues`.
- Sincronizar operaciones con primitivas tales como `Locks`, `Events`, `Semaphores` y `Conditions`.

La biblioteca ofrece tanto primitivas de bajo nivel para usuarios avanzados como abstracciones de alto nivel para facilitar su uso. La clase `Process` es la interfaz principal de bajo nivel para la creación de procesos. Otras clases  de nivel superior, como `Pools`, permiten gestionar de forma sencilla grupos de procesos, evitando las complicaciones propias de la comunicación y sincronización de bajo nivel entre ellos.


---
Parte del código en este notebook está basado en las siguientes fuentes:

- Python 201, Michael Driscoll
- https://www.machinelearningplus.com/python/parallel-processing-python/
- https://github.com/mmckerns/tuthpc
- https://github.com/csc-training/hpc-python
---

### Indentifica el número de núcleos en tu computadora

El número de procesos debería estar relacionado con la cantidad de núcleos físicos de tu equipo.


In [ ]:
# comprueba el número de nucleos
import multiprocessing as mp

# multiprocessing cpu_count solo te proporciona el número de nucleos lógicos (recuerda que los procesadores pueden tener tecnologías como SMT)
print("Number of logical cores: ", mp.cpu_count())

In [ ]:
# utiliza psutil para obtener información más detallada
import psutil

# Número de nucleos lógicos
logical_cores_psutil = psutil.cpu_count(logical=True)

# Número de nucleos físicos
physical_cores_psutil = psutil.cpu_count(logical=False)

# Imprime los resultados
print(f"Logical Cores (psutil): {logical_cores_psutil}")
print(f"Physical Cores (psutil): {physical_cores_psutil}")

## La clase `Process` (paralelismo de procesos a bajo nivel)

La clase `Process` permite crear procesos individuales que llaman a una o varias funciones definidas por el usuario.

Debes crear el proceso y llamar a su método `start()`. Luego, simplemente invoca el método `join()` para todos los procesos que has creado.
Esto es necesario para sincronizar la ejecución de todos tus procesos, esperando a que todos terminen para continuar la ejecución de la aplicación. Si necesitas detener un proceso, puedes llamar a su método `terminate()`.



In [ ]:
from multiprocessing import Process
import os

def doubler(number):
  """
  Función que duplica el valor de entrada y puede ser asignada a un proceso
  """
  result = number * 2
  proc = os.getpid()
  print(f'{number} doubled to {result} by process id: {proc} ')

numbers = [5, 10, 15, 20, 25]

procs_list = []
# cada proceso puede recibir una función a implementar distinta y/o parámetros independientes
for index, number in enumerate(numbers):
  p = Process(target=doubler, args=[number]) # args requiere un elemento iterable
  procs_list.append(p)
  p.start()

# Espera a que todos los procesos terminen
for p in procs_list:
  p.join()


Sin embargo, con este enfoque, obtener los valores de salida de cada proceso requeriría utilizar clases específicas como `multiprocessing.Queue` o `multiprocessing.Manager`. Por lo tanto, se trata de un enfoque complejo, que se utiliza principalmente para generar procesos no relacionados que trabajan de manera independiente (generalmente, no es el caso en la computación científica).

In [ ]:
from multiprocessing import Process

def tarea():
    return 42  # Este valor no se puede recuperar directamente ya que el proceso padre y el hijo no comparten memoria

p = Process(target=tarea)
p.start()
p.join()

El siguiente código muestra cómo el proceso hijo puede enviar un valor al proceso padre utilizando un objeto `Queue`:


In [ ]:
from multiprocessing import Process, Queue

def tarea(q):
    q.put(42)  # Almacete el valor en el objeto de tipo queue

q = Queue()
p = Process(target=tarea, args=(q,))
p.start()
p.join()

resultado = q.get()
print(f'Received value = {resultado}')


NOTA IMPORTANTE: Cuando creas múltiples procesos utilizando `multiprocessing.Process`, cada proceso se ejecuta de forma independiente y posee su propio espacio de memoria. Esto significa que no pueden compartir fácilmente resultados entre sí ni con el proceso principal. Se recomienda utilizar procesos para paralelizar tareas cuando son independientes y no requieren compartir información.

## La clase Pool

**La clase `Pool` se utiliza para representar un conjunto de procesos `worker`**. Tiene métodos que te permiten delegar tareas a los procesos `worker`.

**Es más fácil de trabajar con ella y de un nivel superior que la clase `Process`.**

El proceso Master envía tareas a los workers, los workers ejecutan las tareas y, finalmente, el Master recupera los resultados de los workers.

Los métodos más utilizados en la clase ``Pool`` Class son (aunque existen muchos más):


1. Ejecución síncrona (bloqueante): Los procesos se completan en el mismo orden en el que fueron iniciados. Esto se logra bloqueando el programa principal hasta que los respectivos procesos hayan finalizado.
  - ``Pool.map()`` and ``Pool.starmap()``
  - ``Pool.apply()``


2. Ejecución asíncrona (no bloqueante): El proceso principal no se bloquea. Como resultado, el orden de los resultados puede alterarse, pero generalmente se realiza el trabajo de forma más rápida.
  - ``Pool.map_async()`` and ``Pool.starmap_async()``
  - ``Pool.apply_async()``

El método `map` es aplicable cuando el proceso que vamos a crear se mapea a una función que acepta un único argumento. Para funciones que necesitan múltiples argumentos, se debe utilizar el método `starmap` en su lugar. Ambas versiones reciben un iterable y lo dividen en tareas, donde cada tarea tiene la misma función objetivo (mapeada).

Con respecto a `apply` y `apply_async`, ambos reciben un argumento `args` que acepta los parámetros que se le pasan a la función que se va a asignar a las tareas (la que estamos paralelizando) como argumento, a diferencia de map y de forma similar a starmap. Sin embargo en el caso de `apply` solo realiza una única llamada a la función a paralelizar. ¿Qué significa esto? Para paralelizar realmente la función tienes que iterar manualmente haciendo varias llamadas a `apply` para aprovechar el `pool` de workers. Esto tiene la ventaja de que, en cada llamada, puedes especificar no solo una nueva porción de datos sobre la que trabajará el worker, sino también una tarea completamente diferente (otra función) a ejecutar en ese worker. Es decir, puedes pasar una lista de tareas y una lista de datos con los argumentos para cada tarea.

Para más info sobre apply_async ver:

https://stackoverflow.com/questions/53035293/purpose-of-multiprocessing-pool-apply-and-multiprocessing-pool-apply-async

https://stackoverflow.com/questions/52985131/how-to-write-a-multithreaded-function-for-processing-different-tasks-concurrentl/52992065#52992065

https://docs.python.org/3.8/library/multiprocessing.html#multiprocessing.pool.Pool.apply_async

En tareas intensivas en cómputo, normalmente el problema consiste en aplicar la misma función a una gran cantidad de datos; por ello, en esta práctica nos centraremos en `map` y a los métodos derivados.

Otros métodos el paquete `Multiprocessing` permiten la creación de `pipes`, `queues` y otros enfoques, **pero el `pool` de workers es, con diferencia, el enfoque más típico y sencillo para paralelizar entre los núcleos de un solo ordenador.**




In [ ]:
# Tabla resumen de los métodos de paquete Pool
# ----------------------------------------------------------------------------------------------
#                           |          Una única función              |  Funciones múltiples   |
# ----------------------------------------------------------------------------------------------
#                           |  Un Argumento    |  Varios Argumentos   |  Varios Argumentos     |
# ----------------------------------------------------------------------------------------------
# sync process (blocking)   | Pool.map         | Pool.starmap         |  Pool.apply            |
#
# async proc (non-blocking) | Pool.map_async   | Pool.starmap_async   |  Pool.apply_async      |
# ----------------------------------------------------------------------------------------------

### Uso de Pool.map

Todas las funciones `map` de `Multiprocessing` para un `pool` de workers se comportan de manera similar a la función `map` estándar de Python: ejecutan una función especificada para cada elemento en un iterable que reciben como entrada (tanto la función como el iterable):

```
def square(n):
    return n * n

num_list = [1,2,3,4]
result = map(square, num_list)
print('Mapped result is: ', list(result))

Output:
>> Mapped result is:  [1, 4, 9, 16]
```



In [ ]:
import os
import multiprocessing as mp

def doubler(number):
  """
  A doubling function that can be used by a process
  """
  result = number * 2
  proc = os.getpid()
  print(f'{number} doubled to {result} by process id: {proc} ')
  return result

numbers = [5, 10, 15, 20, 25]

# instantiate a pool of 3 processes
pool = mp.Pool(processes=3)
result = pool.map(doubler, numbers)
pool.close()

print(f'input data: {numbers}')
print(f'results are in corresponding order: {result}')

En este ejemplo, la función `pool.map` funciona de la siguiente manera:

1. **Paso 1: Crea un Pool con 3 procesos**
2. **Paso 2: Asignando tareas**
    - El Pool divide la lista [5, 10, 15, 20, 25] en tareas y las asigna a los 3 procesos.
    - Inicialmente, los 3 procesos comienzan a trabajar en los primeros 3 números:
        * *Process 1*: 5
        * *Process 2*: 10
        * *Process 3*: 15
3. **Paso 3: Ejecución Paralela**
    - Los 3 procesos ejecutan la function en paralelo.
    - Cuando un process finaliza, el Pool le asigna el siguiente número disponible:
        * Si *Process 3* finaliza primero, se le asigna el número 20.
        * Si *Process 1* finaliza a continuación, se le asigna el número 25.

4. **Paso 4: Recogida de Resultados**
    - Aunque los procesos pueden finalizar en cualquier orden, `pool.map` garantiza que los resultados se devuelven en el orden correcto:
        * El primer resultado corresponde a 5.
        * El segundo resultado corresponde a 10.
        * El tercer resultado corresponde a 15.
        * El cuarto resultado corresponde a 20.
        * El quinto resultado corresponde a 25.

### Uso de Pool.map_async

`pool.map_async` es la versión asíncrona de `pool.map`. Esto significa:

- **No bloquea el programa principal**:
    * A diferencia de `pool.map`, que espera a que todos los procesos finalicen antes de continuar, `pool.map_async` devuelve inmediatamente un objeto `AsyncResult` y permite que el programa principal continúe ejecutándose.
- **Recuperando resultados**:
    * Para obtener los resultados, debes llamar al método `.get()` en el objeto `AsyncResult`. Este método bloquea el programa principal hasta que todos los procesos hayan finalizado y los resultados estén disponibles.
- **Orden de los resultados**:
    * Aunque `pool.map_async` es asíncrono, garantiza que los resultados se devuelvan en el mismo orden que la lista de entrada. Esto es similar a `pool.map`.



In [ ]:
import os
import multiprocessing as mp
import time

def doubler(number):
  """
  A doubling function that can be used by a process
  """
  result = number * 2
  proc = os.getpid()
  print(f'{number} doubled to {result} by process id: {proc} ')
  return result

numbers = [5, 10, 15, 20, 25]

# instantiate a pool of 3 processes
pool = mp.Pool(processes=3)
result = pool.map_async(doubler, numbers)
pool.close()  # Don't accept more tasks

####### Here we could do some stuff while the processes run in parallel...

pool.join()   # Wait for all processes ending
results = result.get()# recover the real output data from the result object

print(f'input data: {numbers}')
print(f'results are in corresponding order: {results}')

### Uso de Pool.starmap

Con `Pool.starmap`, en lugar de un único parámetro, se pasan múltiples parámetros en forma de tuplas a la función que se ejecuta en paralelo.

Así, al pasar un iterable como `[(1,2), (3,4), ...]` se obtiene `[func(1,2), func(3,4), ...]`.

In [ ]:
import os
import multiprocessing as mp

def doubler_adder(a, b):
  result = a * 2 + b * 2
  proc = os.getpid()
  print(f'{a}  and {b} doubled and added to {result} by process id: {proc} ')
  return result

# numbers = [5, 10, 15, 20, 25]
numbers_in_tuples = [(x,x+1) for x in range(0,10)] # [(0,1), (1,2), ....]

# instantiate a pool of 3 processes
pool = mp.Pool(processes=3)
result = pool.starmap(doubler_adder, numbers_in_tuples)
# with a single argument, starmap could also be used:
# result = pool.starmap(doubler, [(5,), (10,), (15,), (20,), (25,)])
pool.close()

print(f'input data: {numbers_in_tuples}')
print(f'results are in corresponding order: {result}')

Nota: Puedes obtener el mismo efecto utilizando `Pool.map` (en lugar de `Pool.starmap`), si haces el esfuerzo adicional de unir varios argumentos de la función objetivo en un único argumento (data objetct), como en el siguiente ejemplo, donde la función 'doubler_adder' ha sido modificada para aceptar solo 1 parámetro, que es, de hecho, una lista de los dos parámetros originales:

In [ ]:
import os
import multiprocessing as mp

# function modified to take just a single parameter:
def doubler_adder(a):
  result = a[0] * 2 + a[1] * 2
  proc = os.getpid()
  print(f'{a[0]}  and {a[1]} doubled and added to {result} by process id: {proc} ')
  return result

# numbers = [5, 10, 15, 20, 25]
numbers_in_list = [[x,x+1] for x in range(0,10)] # [[0,1], [1,2], ....]
# instantiate a pool of 3 processes
pool = mp.Pool(processes=3)
result = pool.map(doubler_adder, numbers_in_list)
pool.close()

print(f'input data: {numbers_in_list}')
print(f'results are in corresponding order: {result}')

## Comparando Tiempos de Ejecución

Los siguientes scripts comparan el enfoque bloqueante **blocking multiprocess** con el no bloqueante **non-blocking multiprocess** y la solución con un único proceso **single-process**, midiendo el tiempo de ejecución.

### 1. Enfoque simple:

Sin ganancia de rendimiento en la ejecución en paralelo para tareas limitadas por I/O, tareas simples o pocas tareas.


In [ ]:
import multiprocessing as mp
import time
import numpy as np

def f(x, y):
    return (x+y)**(2)

# generate 2 arrays of 1 million random integers between 1 and 10
x = np.random.randint(1,10,1000000)
y = np.random.randint(1,10,1000000)
print(f'first values in x: {x[0:10]}')
print(f'first values in y: {y[0:10]}')

xy_tuple = [(int(x[i]),int(y[i])) for i in range(0,len(x))]
print(f'first values in xy_tuple: {xy_tuple[0:10]}')

In [ ]:
# generate a 2-process pool.starmap
pool = mp.Pool(2)
# Blocking multiprocess execution
t0 = time.time()
result1 = pool.starmap(f, xy_tuple)
t1 = time.time()
pool.close()
# print results
print(f'first values in result1: {result1[0:10]}')
print(f'time for pool.starmap: {t1-t0}')

In [ ]:
# generate a 2-process pool.starmap_async
pool = mp.Pool(2)
# Non-blocking multiprocess execution "in the background"
t0 = time.time()
result2_ = pool.starmap_async(f, xy_tuple)
pool.close()  # Don't accept more tasks

####### Here we could do some stuff while the processes run in parallel...

pool.join()   # Wait for all the process ending
result2 = result2_.get()# recover the real output data from the result object
t1 = time.time()
# print results
print(f'first values in result2: {result2[0:10]}')
print(f'time for pool.starmap_async: {t1-t0}')

In [ ]:
# Compare with the single-process solution: send data sequentially
result3 = np.zeros(len(x), dtype=int)
#result3 = np.zeros(len(x))
t0 = time.time()
for i in range(len(x)):
  result3[i] = f(x[i],y[i])
t1 = time.time()
print(f'first values in result3: {result3[0:10]}')
print(f'time for single-process: {t1-t0}')

In [ ]:
# Compare with the single-process solution: using vectorized operators
t0 = time.time()
result3 = f(x,y)
t1 = time.time()
print(f'first values in result3: {result3[0:10]}')
print(f'time for single-process: {t1-t0}')

Compara y analiza los tiempos obtenidos al ejecutar las diferentes versiones de los programas.

- A primera vista, se esperaría que el tiempo de ejecución de las versiones paralelas fuese la mitad del requerido por la versión secuencial de un solo proceso del programa. ¿Has observado tal reducción en las mediciones de tiempo? Si no, ¿cuál es la razón?

- Al comparar los tiempos de ejecución de las versiones paralelas sincrónica y asíncrona, ¿qué conclusiones puedes sacar?

- ¿Qué versión del programa es más rápida? ¿Cuáles son las razones que hacen de esta versión la más optimizada?

> **NOTA IMPORTANTE**
> - Una versión paralela de un programa puede ser más lenta que una secuencial. Esto puede ocurrir cuando existe una gran sobrecarga debido al envío de datos a cada proceso.
> - Cuando defines tareas relativamente simples (x+y)<sup>2</sup>, el tiempo empleado en enviar datos y recoger resultados es mucho mayor que el tiempo dedicado al cálculo real.
> - Para aprovechar el procesamiento paralelo, debemos mantener la pool de procesos ocupada con tareas relativamente complejas y minimizar la sobrecarga de comunicación. Y, por supuesto, debemos usar un mayor número de procesadores físicos.

### 2. Versión data-chunked.

Como demostración, consulta el siguiente código, ligeramente modificado respecto al ejemplo anterior:

**NOTA**:

También haremos uso de los bloques `with`, conocidos como **context managers**. Un **context manager** es un constructo que permite asignar y liberar recursos automáticamente al entrar y salir de un bloque de código. La sentencia `with` asegura que las operaciones de setup y cleanup se gestionen correctamente, incluso si se producen excepciones dentro del bloque. En este caso, elimina la necesidad de llamar explícitamente a `pool.close()` y `pool.join()` al final de la sección paralela.

El uso de **context managers** se recomienda generalmente en la programación en Python, pero es particularmente importante en la programación paralela.



In [ ]:
# This example defines a compute-intensive function and sends data in chunks
# NOTE: Recommended to test this example in a (virtual) machine with 4 logical cores.
import time
import numpy as np
import multiprocessing as mp

# Generate 40 million random floats
N = 40_000_000
x = np.random.rand(N).astype(np.float64)

# Create 4 chunks of 10 million elements each:
chunk_size = 10_000_000
chunks = [x[i:i + chunk_size] for i in range(0, N, chunk_size)]
print(f"Number of chunks: {len(chunks)}")
print(f"Size of each chunk: {chunk_size}")

# Define a CPU-intensive function
def heavy_function(array):
    for _ in range(3):
        array = (np.sin(array)+np.cos(array))**(array*array)
    return np.sum(array)  # Just return the sum of the processed chunk

# Multiprocessing - blocking version (Pool.map)
def parallel_map(chunks_list, num_procs=4):
    with mp.Pool(processes=num_procs) as pool:
        results = pool.map(heavy_function, chunks_list)
    return results

# Multiprocessing - async version (Pool.map_async)
def parallel_map_async(chunks_list, num_procs=4):
    with mp.Pool(processes=num_procs) as pool:
        async_result = pool.map_async(heavy_function, chunks_list)
        results = async_result.get()  # Wait for processes to finish
    return results

# Single-process (serial) execution
def serial_execution(chunks_list):
    results = []
    for chunk in chunks_list:
        results.append(heavy_function(chunk))
    return results

# time all three approaches:
# execute and time Parallel blocking code (Pool.map)
start_time = time.time()
results_map = parallel_map(chunks)
end_time = time.time()
print(f"[Pool.map]     Elapsed time: {end_time - start_time:.2f} seconds")

# execute and time Parallel async code (Pool.map_async)
start_time = time.time()
results_map_async = parallel_map_async(chunks)
end_time = time.time()
print(f"[map_async]    Elapsed time: {end_time - start_time:.2f} seconds")

#execute and time Serial code
start_time = time.time()
results_serial = serial_execution(chunks)
end_time = time.time()
print(f"[Single-process] Elapsed time: {end_time - start_time:.2f} seconds")


### 3. Otro ejemplo: descomposición en factores primos

Un nuevo ejemplo, en este caso utilizando una tarea más compleja: la descomposición en factores primos de números grandes.

En este caso, también estamos utilizando `tqdm` para proporcionar una barra de progreso que soporta tanto la ejecución single-process como la multi-process.


In [ ]:
# This example uses a function that computes prime factors of a number
# NOTE: On a machine with just 1 physical core there won't be performance
# gains, use an engine with more cores to perceive the gain in this example

import time
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm

# Define a function to decompose a number into its prime factors
def prime_factors(n):
    factors = []
    divisor = 2
    while n > 1:
        while n % divisor == 0:
            factors.append(divisor)
            n //= divisor
        divisor += 1
        if divisor * divisor > n and n > 1:
            factors.append(n)
            break
    return factors

# Create data to process (large numbers for factorization)
# create a list of 100 random floating point numbers between 1e14 and 1e18
data = np.random.uniform(1e10, 1e12, 100)

# Single-process execution
def single_process_execution(data):
    results = []
    for number in tqdm(data, desc="Single-process execution"):
        results.append(prime_factors(number))
    return results

# Multi-process execution with starmap
def multi_process_execution(data):
    with Pool(2) as pool:  # Use 2 processes
        results = list(tqdm(pool.map(prime_factors, data), total=len(data), desc="Multi-process execution"))
    return results

# Measure time for single process execution
start_time = time.time()
single_results = single_process_execution(data)
single_duration = time.time() - start_time
print(f"Single-process execution time: {single_duration:.2f} seconds")

# Measure time for multi-process execution
start_time = time.time()
multi_results = multi_process_execution(data)
multi_duration = time.time() - start_time
print(f"Multi-process execution time (2 cores): {multi_duration:.2f} seconds")

# Verify that the results are identical
assert single_results == multi_results, "Results do not match!"
print("Results are identical.")


## Ejecutando varios scripts python en paralelo

In [ ]:
%%file script1.py
import os
print(f'hello from script 1, executed by process {os.getpid()}.')
f= open("file1.txt","w+")
f.write("hello from script 1")
f.close()


In [ ]:
%%file script2.py
import os

print(f'hello from script 2, executed by process {os.getpid()}.')
f= open("file2.txt","w+")
f.write("hello from script 2")
f.close()

In [ ]:
%%file script3.py
import os

print(f'hello from script 3, executed by process {os.getpid()}.')
f= open("file3.txt","w+")
f.write("hello from script 3")
f.close()

 Now, run 3 processes so that each process executes one of the python scripts in parallel with the other:

In [ ]:
import os
import multiprocessing as mp
import subprocess

script_list = ['script1.py', 'script2.py', 'script3.py']

def run_python(process):
  result = subprocess.run(["python", process], capture_output=True, text=True)
  return result.stdout

pool = mp.Pool(processes=3)
results = pool.map(run_python, script_list)
pool.close()

print(f'results = {results}')

## Uso de Pool.apply

``Pool.map`` y ``Pool.apply`` bloquearán el programa principal hasta que todos los procesos hayan finalizado, lo cual es muy útil si queremos obtener resultados en un orden particular para ciertas aplicaciones.

En contraste, las variantes async enviarán todos los procesos a la vez y recuperarán los resultados tan pronto como hayan finalizado. Otra diferencia es que necesitamos usar el método get después de la llamada a apply_async() para obtener los valores de retorno de los procesos finalizados.

El orden de los resultados no está garantizado que sea el mismo que el orden de las llamadas a ``Pool.apply_async``.

Observa también que se podrían llamar a varias funciones diferentes con ``Pool.apply_async`` (no todas las llamadas necesitan usar la misma función). En contraste, ``Pool.map`` aplica la misma función a muchos argumentos.

In [ ]:
from multiprocessing import Pool

def doubler(number):
  """
  A doubling function that can be used by a process
  """
  result = number * 2
  proc = os.getpid()
  print(f'{number} doubled to {result} by process id: {proc} ')
  return result

numbers = [5, 10, 15, 20, 25]

results =[]
pool = Pool(processes=3)
for i,number in enumerate(numbers): # note the for-loop!!! calling apply_async creates just a single process
  results.append(pool.apply_async(doubler, (numbers[i],)).get(timeout=1))

print(results)
